<span style="font-size: 2em; font-weight:bold">AI 70's Country</span>

In [24]:
import warnings
warnings.filterwarnings('ignore')

import scipy
import numpy as np
import matplotlib
import pandas as pd
import statsmodels
import sklearn
import tensorflow
#import keras

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense,LSTM,Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import History, EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.constraints import maxnorm
import string


import json
import time

As always, data prep is the hardest part of the project.  Because I am going to use a validation set in training my model, and because Keras uses the last n% of the data as the validation set, I want to shuffle the lyrics so that my validation set contains a better representation of all the data - not just the last song.  I also want to get the most originality that I can out of the model, so I will eliminate duplicate lyrics (some songs have refrains that repeat multiple times).  

In [4]:
lyricsCSV = pd.read_csv('lyricsTrain_35.csv',encoding='ISO-8859-1')
lyricsCSV.sort_values(lyricsCSV.columns[0],inplace=True)
lyricsCSV.drop_duplicates(keep='first',inplace=True)
lyricsCSV = lyricsCSV.sample(frac=1)



lyricsCSV.to_csv('lyrics.txt',sep='\t',index=False)
l = open('lyrics.txt','r')
lyrics = l.read()
l.close()

Remove lines and print

In [5]:
tokens = lyrics.split()
lyrics = ' '.join(tokens)
print(lyrics)

lyrics That makes a body feel alone And thank God you're a country boy There's nothin' left for me to say-ay Cause she walks dowtown with a suitcase in her hand How you could easily take my man If I were a carpenter Your beauty is beyond compare You put me high upon a pedestal My daddy worked all night in the Van Lear coal mines But if that's what it takes to hold you Hope you're doin' fine Just for old time's sake Tell me you love me and don't let me cry Say anything but don't say goodbye And I stopped beside the Sunday school As the tears fell on his daddy's face, he heard these words again People writing letters back home To lose these memories It's too sad to write My happiness depends on you Well, my fiddle was my daddy's till the day he died And I can't believe it's you Thank God I'm a country boy """Ninety days, Jerry, when you hot, you hot""" Who claims that he just don't believe in fightin' And there's nothing I can do to keep A lotta sad people thinkin' that's mighty keen """

One final thing, when I look at the above lyrics, I seem to see a LOT of quotation marks.  So, I am going to just replace those with a space.  

In [6]:
lyrics = lyrics.replace('"',' ')

Now, we can build sequences of characters that will be used to predict a final character

In [7]:
length = 100 # Length of the characer sequences (because we have so much verbage,
             # we can use a relatively large number)
sequences = list()
for i in range(length, len(lyrics)):
    seq = lyrics[i-length:i+1]
    sequences.append(seq)

Create and save a .txt file of our sequences with line endings

In [8]:
data = '\n'.join(sequences)
file = open('char_sequences.txt','w')
file.write(data)
file.close()

Create a dictionary of character:number mappings 

In [9]:
file = open('char_sequences.txt','r')
raw_text = file.read()
file.close()

lines = raw_text.split('\n')

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

# Save the mapping as json 
json_map = json.dumps(mapping)
__ = open('mapping.json','w')
__.write(json_map)
__.close()

Use the dictionary to create sequences of numbers only (numbers that describe the characters)

In [10]:
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)
    


Create input sets (with 99 characters) and output sets (1 character) and then one-hot code the sets so we can use them to train the model.

In [11]:
vocab_size = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] #one-hot code input
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) #one-hot code output

Fit the model with tuning parameters determined by trial and error.

In [10]:

def countryTrain(paramUnits,paramEpochs,paramValSplit,paramShuffle,paramBatchSize,paramDropout):

    #Define callbacks
    es = EarlyStopping(monitor = 'acc',min_delta = .01, patience = 2, mode = 'max',verbose=1)
    mc = ModelCheckpoint('model.drop_{}.best'.format(str(paramDropout)), monitor='loss', mode='min', save_best_only=True) # Keep best model


    # define and fit model
    model = Sequential()
    model.add(LSTM(paramUnits, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(paramDropout))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    modelLyrics = model.fit(X, y, epochs = paramEpochs, validation_split = paramValSplit, 
                            shuffle = paramShuffle, batch_size = paramBatchSize, verbose=1,callbacks=[es,mc])

   
    history = pd.DataFrame(modelLyrics.history)
    history.to_csv('modelLyricsHistory_drop_{}.csv'.format(str(paramDropout)),index=False)



In [25]:
units = 512  # From the mentioned article in data science
epochs = 100  # Just a large number since I am using early stopping
validationSplit = 0.1 # My data set is small so I want to use as much as possible to train vs. validate
shuffle = False
batchSize = 32 # Doubled the default batch size to speed up training
dropOut = .5  # http://papers.nips.cc/paper/4878-understanding-dropout.pdf
    #Define callbacks

mc = ModelCheckpoint('model.100Epoch_6.6.best', monitor='acc', mode='max', save_best_only=True) # Keep best model


    # define and fit model
model = Sequential()
model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(dropOut))
model.add(Dense(vocab_size, activation='softmax',kernel_constraint=maxnorm(3)))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


modelLyrics = model.fit(X, y, epochs = epochs, validation_split = validationSplit, 
                            shuffle = shuffle, batch_size = batchSize, verbose=1,callbacks=[mc])

   
history = pd.DataFrame(modelLyrics.history)
history.to_csv('modelLyricsHistory.100Epoch_6.6.2.csv',index=False)

Train on 26265 samples, validate on 2919 samples
Epoch 1/100
26265/26265 [==============================] - 189s 7ms/sample - loss: 2.7718 - acc: 0.2607 - val_loss: 2.3467 - val_acc: 0.3570
Epoch 2/100
26265/26265 [==============================] - 188s 7ms/sample - loss: 2.3820 - acc: 0.3552 - val_loss: 2.1293 - val_acc: 0.4018
Epoch 3/100
26265/26265 [==============================] - 187s 7ms/sample - loss: 2.1284 - acc: 0.4005 - val_loss: 1.9944 - val_acc: 0.4282
Epoch 4/100
26265/26265 [==============================] - 187s 7ms/sample - loss: 1.9806 - acc: 0.4327 - val_loss: 1.8957 - val_acc: 0.4532
Epoch 5/100
26265/26265 [==============================] - 187s 7ms/sample - loss: 1.8626 - acc: 0.4654 - val_loss: 1.8591 - val_acc: 0.4618
Epoch 6/100
26265/26265 [==============================] - 188s 7ms/sample - loss: 1.7549 - acc: 0.4931 - val_loss: 1.8192 - val_acc: 0.4666
Epoch 7/100
26265/26265 [==============================] - 188s 7ms/sample - loss: 1.6467 - acc: 0.5200 -

26265/26265 [==============================] - 186s 7ms/sample - loss: 1.1493 - acc: 0.6626 - val_loss: 1.7804 - val_acc: 0.5050
Epoch 59/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.1432 - acc: 0.6658 - val_loss: 1.8013 - val_acc: 0.4896
Epoch 60/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.1106 - acc: 0.6769 - val_loss: 1.8047 - val_acc: 0.4998
Epoch 61/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.1787 - acc: 0.6529 - val_loss: 1.8210 - val_acc: 0.4865
Epoch 62/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.3081 - acc: 0.6162 - val_loss: 1.8234 - val_acc: 0.4741
Epoch 63/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.3508 - acc: 0.5998 - val_loss: 1.7695 - val_acc: 0.4926
Epoch 64/100
26265/26265 [==============================] - 186s 7ms/sample - loss: 1.2800 - acc: 0.6263 - val_loss: 1.7839 - val_acc: 0.4947
Epoch 65/100
26265/

In [11]:
#Define parameters  -
units = 512  # From the mentioned article in data science
epochs = 100  # Just a large number since I am using early stopping
validationSplit = 0.1 # My data set is small so I want to use as much as possible to train vs. validate
shuffle = True
batchSize = 64 # Doubled the default batch size to speed up training
dropOut = .5  # http://papers.nips.cc/paper/4878-understanding-dropout.pdf


for i in range(2,6):
    d = i*0.1
    countryTrain(units,epochs,validationSplit,shuffle,batchSize,d)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 20118 samples, validate on 2236 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
20118/20118 [==============================] - 76s 4ms/sample - loss: 2.9322 - acc: 0.2276 - val_loss: 2.5897 - val_acc: 0.3032
Epoch 2/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 2.4171 - acc: 0.3365 - val_loss: 2.3031 - val_acc: 0.3582
Epoch 3/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 2.2339 - acc: 0.3676 - val_loss: 2.1955 - val_acc: 0.3913
Epoch 4/100
20118/20118 [==============================] - 75s 4ms/sample - loss: 2.1129 - acc: 0.4010 - val_loss: 2.1359 - val_acc: 0.3971
Epoch 5/100
20118/20118 [==============================] - 75s 4ms/sample - loss: 2.0526 - acc: 0.4241 - val_loss: 2.1109 - val_acc: 0.4191
Epoch 6/100
2011

20118/20118 [==============================] - 74s 4ms/sample - loss: 0.3969 - acc: 0.8775 - val_loss: 2.4419 - val_acc: 0.4799
Epoch 27/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 0.3617 - acc: 0.8875 - val_loss: 2.4361 - val_acc: 0.4821
Epoch 28/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 0.3303 - acc: 0.8973 - val_loss: 2.5147 - val_acc: 0.4741
Epoch 29/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 0.3063 - acc: 0.9073 - val_loss: 2.5394 - val_acc: 0.4803
Epoch 30/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 0.2937 - acc: 0.9119 - val_loss: 2.5923 - val_acc: 0.4781
Epoch 31/100
20118/20118 [==============================] - 75s 4ms/sample - loss: 0.2725 - acc: 0.9186 - val_loss: 2.6167 - val_acc: 0.4758
Epoch 32/100
20118/20118 [==============================] - 74s 4ms/sample - loss: 0.2636 - acc: 0.9185 - val_loss: 2.6633 - val_acc: 0.4767
Epoch 33/100
20118/20118 [

In [12]:
# store history

Create a function that encodes a kickoff text string and then plugs it into our trained model

In [16]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_lyric, n_chars):
    lyrics = seed_lyric
    for __ in range(n_chars):
    # encode the characters as integers
        encoded = [mapping[char] for char in lyrics]
    # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
        yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
    # append to input
        lyrics += char
    return lyrics

Is there a song other than Stairway to Heaven that I could have used for the kickoff sequence?

In [21]:
startLyrics = "I've had a largemouth bass bust my line A couple beautiful girls tell me, Goodbye Trucks break down,"

In [19]:
startLyrics = "There's a lady who's sure All that glitters is gold And she's buying a stairway to heaven When she g"

In [27]:
__ = open('lyrics.Coolio.Epoch100.txt','w')

model = load_model('model.100Epoch_6.6.best')
lyricsFinal = generate_seq(model,mapping,length,startLyrics,1000)
__ = open('lyrics.Coolio.Epoch100.txt','a+')
#__.write('Drop {}\n\n'.format(modelNum))
__.write(lyricsFinal)
__.write('.\n\n\n')
__.close()

Run the model and print the lyrics

In [29]:
def run_models(modelNum):
    model = load_model('model.drop_0.{}.best'.format(modelNum))
    lyricsFinal = generate_seq(model,mapping,length,startLyrics,1000)
    __ = open('lyrics.LedZep.txt','a+')
    __.write('Drop {}\n\n'.format(modelNum))
    __.write(lyricsFinal)
    __.write('.\n\n\n')
    __.close()
    
    

In [30]:
__ = open('lyrics.LedZep.txt','w')
__.write('Stairway to Heaven/n/n')
__.close()

for i in range(0,6):
    run_models(str(i))

df = pd.read_json('modelLyricsHistory.json')

df